In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Time
import time

# Random and os for reproducibility
import random
import os

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
import tensorflow as tf

# Processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

# Pipeline
from sklearn.pipeline import Pipeline

# Data imputation
from sklearn.impute import SimpleImputer

# Making keras compatible with scikit learn api
# https://scikit-learn.org/stable/developers/develop.html
from sklearn.base import BaseEstimator, RegressorMixin

# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

# Converting Times Series Data to 3D format
from src.utils.split_sequences import split_sequences

# To run cross validation parallelized
from joblib import Parallel, delayed

2024-07-18 22:26:33.989523: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-18 22:26:33.991893: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 22:26:34.039002: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-18 22:26:34.040455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-07-18 22:26:34.833905: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


<IPython.core.display.Javascript object>

# Functions and definitions

## Helper functions for blocked time series cross validation

### Convert train/test data to 3D format

In [3]:
def generate_sequences_helper(
    dataset, cement_types, dates=None, timesteps=None, split_by_cement_type=False
):
    index_train = dataset["y_train"].index
    index_test = dataset["y_test"].index

    dataset["y_train"] = dataset["y_train"].reset_index(drop=True)
    dataset["y_test"] = dataset["y_test"].reset_index(drop=True)

    if dates is not None:
        dataset["dates_train"] = dates[index_train].reset_index(drop=True)
        dataset["dates_test"] = dates[index_test].reset_index(drop=True)

    dataset["cement_types_train"] = cement_types.loc[index_train].reset_index(drop=True)
    dataset["cement_types_test"] = cement_types.loc[index_test].reset_index(drop=True)

    dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

    return dataset

<IPython.core.display.Javascript object>

In [4]:
def generate_sequences(dataset, timesteps, split_by_cement_type=False):
    if split_by_cement_type:
        dataset["x_train"], dataset["y_train"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_train"],
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["cement_types_train"],
                    dataset["y_train"],
                ],
                axis=1,
            ),
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences_per_cement_type(
            pd.concat(
                [
                    dataset["dates_test"],
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["cement_types_test"],
                    dataset["y_test"],
                ],
                axis=1,
            ),
            timesteps,
        )
    else:
        dataset["x_train"], dataset["y_train"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_train"], columns=x.columns),
                    dataset["y_train"],
                ],
                axis=1,
            ).values,
            timesteps,
        )

        dataset["x_test"], dataset["y_test"] = split_sequences(
            pd.concat(
                [
                    pd.DataFrame(dataset["x_test"], columns=x.columns),
                    dataset["y_test"],
                ],
                axis=1,
            ).values,
            timesteps,
        )
    return dataset

<IPython.core.display.Javascript object>

### Data preprocessing

In [5]:
def impute_data(dataset, imputer=None, imputer_params=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply imputation to the data
    if imputer is not None:
        imputer = imputer() if imputer_params is None else imputer(**imputer_params)
        x_train = imputer.fit_transform(x_train)
        x_test = imputer.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [6]:
def transform_data(dataset, transformer=None):
    x_train = dataset["x_train"]
    x_test = dataset["x_test"]

    # Apply data normalization/standardization to the data
    if transformer is not None:
        scaler = transformer()
        x_train = scaler.fit_transform(x_train)
        x_test = scaler.transform(x_test)

    dataset["x_train"] = x_train
    dataset["x_test"] = x_test

    return dataset

<IPython.core.display.Javascript object>

In [7]:
def preprocess_data(dataset, transformer=None, imputer=None, imputer_params=None):
    dataset = impute_data(dataset, imputer, imputer_params)
    dataset = transform_data(dataset, transformer)
    return dataset

<IPython.core.display.Javascript object>

### Train and evaluate the model

In [8]:
def train_and_evaluate_model(Estimator, dataset, estimator_params=None):
    """
    Purpose: Helper function to be used in conjunction with
    blocked time_series cross validation function
    """
    x_train = dataset["x_train"]
    y_train = dataset["y_train"]
    x_test = dataset["x_test"]
    y_test = dataset["y_test"]

    # Instantiate the model
    model = Estimator() if estimator_params is None else Estimator(**estimator_params)

    # Fitting the model
    model.fit(x_train, y_train)

    # Making predictions on train/test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Return regression metrics
    return score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

<IPython.core.display.Javascript object>

### Custom Cross Validate

In [9]:
def custom_cross_validate_parallelized(
    Estimator,
    Imputer,
    Transform,
    x,
    y,
    cv,
    timesteps,
    dates=None,
    cement_types=None,
    estimator_params=None,
    imputer_params=None,
    split_by_cement_type=True,
    n_jobs=-1,  # Set the number of parallel jobs, -1 for using all available cores
):
    def process_fold(train_index, test_index, dates, cement_types, x, y):
        dataset = {
            "dates_train": dates[train_index].reset_index(drop=True),
            "cement_types_train": cement_types.loc[train_index].reset_index(drop=True),
            "x_train": x.loc[train_index].reset_index(drop=True),
            "y_train": y[train_index].reset_index(drop=True),
            "dates_test": dates[test_index].reset_index(drop=True),
            "cement_types_test": cement_types.loc[test_index].reset_index(drop=True),
            "x_test": x.loc[test_index].reset_index(drop=True),
            "y_test": y[test_index].reset_index(drop=True),
        }

        set_seeds(SEED + REPEAT)

        # Preprocess the dataset
        dataset = preprocess_data(dataset, Transform, Imputer, imputer_params)

        # generate sequences (3D format)
        dataset = generate_sequences(dataset, timesteps, split_by_cement_type)

        # Train and Evaluate the model
        score = train_and_evaluate_model(Estimator, dataset, estimator_params)
        return score

    scores = Parallel(n_jobs=n_jobs)(
        delayed(process_fold)(train_index, test_index, dates, cement_types, x, y)
        for train_index, test_index in cv.split(x)
    )

    # After every iteration metrics results are appended together
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += [value]

    results = [scores_final]
    return results

<IPython.core.display.Javascript object>

## Model Definition

In [10]:
class LSTM(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.model = self.get_model()
        self.batch_size = 16
        self.epochs = 300
        self.verbose = 0

    def fit(self, X=None, y=None):
        self.model.fit(
            X, y, batch_size=self.batch_size, epochs=self.epochs, verbose=self.verbose
        )

    def predict(self, X=None):
        return self.model.predict(X, verbose=self.verbose)

    def get_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.LSTM(units=16, activation="relu"))
        model.add(tf.keras.layers.Dropout(rate=0.10))
        model.add(tf.keras.layers.Dense(units=1))
        model.compile(
            # optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001),
            loss="mse",
            metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
        )
        return model

<IPython.core.display.Javascript object>

In [11]:
def pad_time_series(dataframe, timesteps):
    """
    Pad timeseries with zeros
    """
    df_tmp = pd.DataFrame(
        dict(
            zip(
                dataframe.columns,
                [[0 for _ in range(timesteps - 1)] for _ in range(dataframe.shape[1])],
            )
        )
    )
    df_tmp[DATE] = dataframe[DATE].iloc[0]
    return pd.concat([df_tmp, dataframe], axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

In [12]:
def split_sequences_per_cement_type(dataframe, timesteps, pad=False):
    """
    Create sequences per cement time
    to avoid having parts of the sequence
    of different types of cement.
    """
    if timesteps == 1:
        return split_sequences(
            dataframe.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps
        )

    dates = dataframe[DATE][timesteps - 1 :]
    data = []
    dataframes = []

    for cement_type in CEMENT_TYPES:
        data.append(dataframe[dataframe[cement_type] == 1])
    data.append(dataframe[(dataframe[CEMENT_TYPES] == 0).all(axis=1)])

    for df in data:
        if pad:
            dates = df[DATE].reset_index(drop=True)
            df = pad_time_series(df, timesteps).reset_index(drop=True)
        else:
            dates = df[DATE][timesteps - 1 :].reset_index(drop=True)
        x, y = split_sequences(df.drop([DATE] + CEMENT_TYPES, axis=1).values, timesteps)
        x = pd.DataFrame({"Sequences": [sample.tolist() for sample in x]})
        y = pd.DataFrame({"Target": y})
        dataframes.append(pd.concat([dates, x, y], axis=1))

    data = pd.concat(dataframes, axis=0)
    data[DATE] = pd.to_datetime(data[DATE])
    data = data.sort_values(by=DATE).reset_index(drop=True)
    x = data["Sequences"]
    y = data["Target"].values
    x = np.array(x.tolist())

    return x, y

<IPython.core.display.Javascript object>

# Settings for Reproducibility

In [13]:
SEED = 47
REPEAT = 0

<IPython.core.display.Javascript object>

In [14]:
def set_seeds(seed=SEED):
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)


# https://stackoverflow.com/questions/36288235/how-to-get-stable-results-with-tensorflow-setting-random-seed

<IPython.core.display.Javascript object>

In [15]:
def set_global_determinism():
    set_seeds(seed=SEED)

    os.environ["TF_DETERMINISTIC_OPS"] = "1"
    os.environ["TF_CUDNN_DETERMINISTIC"] = "1"

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

<IPython.core.display.Javascript object>

In [16]:
def suppress_warnings():
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

<IPython.core.display.Javascript object>

In [17]:
def supress_other_stuff():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    # os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

<IPython.core.display.Javascript object>

In [18]:
suppress_warnings()
supress_other_stuff()

<IPython.core.display.Javascript object>

In [19]:
index_to_save = 17

<IPython.core.display.Javascript object>

In [20]:
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}
DATE = "Date"
CEMENT_TYPES = ["Cement_Type_Type III", "Cement_Type_Type IL"]

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [21]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + Feature Engineering",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "LSTM",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [22]:
df = pd.read_csv("../../../../../../data/processed/partner_ii/cement-shipping.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [23]:
df_copy = df.copy()
df_copy = pd.get_dummies(data=df_copy, columns=["Cement_Type"], drop_first=True)
df_copy = df_copy.drop(
    [
        # Properties
        "Initial setting time",
        "Blaine",
        "Sieve 32 um",
        "Sieve 45 um",
        "CS1",
        "CS3",
        "CS7",
    ],
    axis=1,
).copy()

<IPython.core.display.Javascript object>

In [24]:
df_copy

,Date,CaO,SiO2,Al2O3,Fe2O3,MgO,SO3,K2O,Na2O,Total C3S,Total C2S,C3A,C4AF,Free CaO,Loss on Ignition,CS28,Cement_Type_Type III,Cement_Type_Type IL
0,2020-01-01 11:00:00+00:00,63.45,19.63,4.90,3.30,3.50,2.89,0.65,0.06,57.88,14.47,5.55,11.26,0.67,3.08,42.93,False,False
1,2020-01-02 11:00:00+00:00,62.22,19.30,4.80,3.23,3.60,3.20,0.67,0.07,57.86,14.79,5.33,11.46,0.69,3.08,49.72,True,False
2,2020-01-02 11:00:00+00:00,62.34,19.28,4.81,3.29,3.48,2.85,0.66,0.06,57.24,14.72,5.52,11.06,0.75,3.38,39.49,False,False
3,2020-01-03 11:00:00+00:00,62.18,19.29,4.85,3.33,3.47,2.87,0.69,0.08,58.03,13.46,5.55,11.33,0.70,3.20,40.55,False,False
4,2020-01-04 11:00:00+00:00,61.97,19.28,4.93,3.39,3.49,2.87,0.72,0.11,58.32,13.38,5.73,11.56,0.63,2.94,42.64,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,2021-12-28 11:00:00+00:00,63.32,19.52,4.39,3.25,2.26,3.39,0.61,0.09,59.16,14.46,4.62,12.13,0.64,2.86,46.44,True,False
1049,2021-12-29 11:00:00+00:00,62.24,18.11,4.19,3.03,2.26,3.13,0.59,0.07,52.90,15.11,4.59,10.77,0.76,5.56,39.87,False,True
1050,2021-12-30 11:00:00+00:00,62.23,18.10,4.21,3.03,2.26,3.08,0.59,0.08,52.27,15.41,4.74,10.42,0.85,5.60,36.74,False,True
1051,2021-12-31 11:00:00+00:00,62.27,18.02,4.15,2.99,2.33,3.12,0.60,0.08,53.11,14.65,4.49,10.41,1.05,5.73,36.36,False,True


<IPython.core.display.Javascript object>

## Feature Engineering

In [25]:
# Feature Engineering over Chemical Features
ch_features = ["CaO", "MgO", "Na2O", "Al2O3", "SiO2", "SO3", "K2O", "Fe2O3"]

df_copy["std_ch_feats"] = df_copy[ch_features].std(ddof=0, axis=1)

df_copy["ratio_CaO_to_SiO2"] = df_copy["CaO"] / df_copy["SiO2"]
df_copy["ratio_MgO_to_CaO"] = df_copy["MgO"] / df_copy["CaO"]

# Feature Engineering over Mineralogical Features
mi_features_set1 = ["C3A", "C4AF"]
df_copy["std_mi_set1_feats"] = df_copy[mi_features_set1].std(ddof=0, axis=1)
df_copy["ratio_Aluminate_to_Ferrite"] = df_copy["C3A"] / df_copy["C4AF"]

<IPython.core.display.Javascript object>

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [26]:
df_copy[CEMENT_TYPES] = df_copy[CEMENT_TYPES].astype(float)
dates = df["Date"].copy()
y = df_copy.pop("CS28")
x = df_copy
df_copy = pd.concat([x, y], axis=1)

<IPython.core.display.Javascript object>

In [27]:
TIMESTEPS_LIST = [1, 7, 14]

<IPython.core.display.Javascript object>

# 1. Long Short Term Memory - LSTM

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Timesteps:</b> 1, 3, 5, 7, 10, 15, 20<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [28]:
start = time.time()

repeats = 3
n_splits = 5
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
    x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
    y = df_copy["CS28"]
    scores = custom_cross_validate_parallelized(
        LSTM,
        SimpleImputer,
        StandardScaler,
        x,
        y,
        cv,
        timesteps=timesteps,
        dates=dates,
        cement_types=df_copy[CEMENT_TYPES],
        estimator_params=None,
        imputer_params={"strategy": "median"},
        split_by_cement_type=True,
    )
    scores = scores[0]
    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores, METRICS, METRICS_DICT)

    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Repeated KFold"
    results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Repeated Cross Validation:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.198 (0.039)
MAE: 1.754 (0.030)
MAPE: 0.041 (0.001)
R2: 0.612 (0.015)


******
[TEST]
******
RMSE: 2.448 (0.102)
MAE: 1.930 (0.086)
MAPE: 0.045 (0.002)
R2: 0.510 (0.063)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.777 (0.117)
MAE: 1.431 (0.106)
MAPE: 0.033 (0.002)
R2: 0.742 (0.035)


******
[TEST]
******
RMSE: 2.543 (0.270)
MAE: 1.979 (0.161)
MAPE: 0.046 (0.004)
R2: 0.449 (0.146)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.681 (0.117)
MAE: 1.339 (0.109)
MAPE: 0.031 (0.002)
R2: 0.768 (0.031)


******
[TEST]
******
RMSE: 2.705 (0.255)
MAE: 2.103 (0.156)
MAPE: 0.049 (0.004)
R2: 0.329 (0.151)


Minutes Elapsed:  6.612322251001994


<IPython.core.display.Javascript object>

In [29]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model Cross Validation  \
                                                                           
0  Chemical + Mineralogical + Feature Engineering  LSTM   Repeated KFold   
1  Chemical + Mineralogical + Feature Engineering  LSTM   Repeated KFold   
2  Chemical + Mineralogical + Feature Engineering  LSTM   Repeated KFold   

  Timesteps RMSE Test            MAE Test           MAPE Test            \
                 mean       std      mean       std      mean       std   
0         1  2.447537  0.102006  1.929530  0.085710  0.044662  0.001983   
1         7  2.543129  0.269894  1.978938  0.161019  0.046151  0.003689   
2        14  2.705009  0.254875  2.102688  0.155796  0.049098  0.003892   

    R2 Test            
       mean       std  
0  0.510017  0.063026  
1  0.449090  0.145756  
2  0.329255  0.150663

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [30]:
start = time.time()

repeats = 3
n_splits = 5
train_size = 0.8
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)

        scores = custom_cross_validate_parallelized(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
            split_by_cement_type=True,
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "train_size": 0.8}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 4.429 (0.428)
MAE: 3.402 (0.318)
MAPE: 0.079 (0.007)
R2: -1.045 (0.413)


******
[TEST]
******
RMSE: 12.299 (2.932)
MAE: 9.335 (2.162)
MAPE: 0.217 (0.056)
R2: -15.614 (8.336)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 2.021 (0.539)
MAE: 1.634 (0.458)
MAPE: 0.038 (0.011)
R2: 0.508 (0.286)


******
[TEST]
******
RMSE: 9.779 (3.836)
MAE: 7.822 (3.255)
MAPE: 0.185 (0.087)
R2: -22.762 (32.120)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 2.184 (0.659)
MAE: 1.771 (0.572)
MAPE: 0.041 (0.013)
R2: 0.371 (0.476)


******
[TEST]
******
RMSE: 12.625 (8.583)
MAE: 11.251 (8.124)
MAPE: 0.268 (0.192)
R2: -69.728 (74.198)


Minutes Elapsed:  2.572239871819814


<IPython.core.display.Javascript object>

In [31]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model  \
                                                          
0  Chemical + Mineralogical + Feature Engineering  LSTM   
1  Chemical + Mineralogical + Feature Engineering  LSTM   
2  Chemical + Mineralogical + Feature Engineering  LSTM   
3  Chemical + Mineralogical + Feature Engineering  LSTM   
4  Chemical + Mineralogical + Feature Engineering  LSTM   
5  Chemical + Mineralogical + Feature Engineering  LSTM   

             Cross Validation Timesteps  RMSE Test             MAE Test  \
                                              mean       std       mean   
0  Blocking Time Series Split         1  12.299179  2.931981   9.335137   
1  Blocking Time Series Split         7   9.779360  3.836300   7.821624   
2  Blocking Time Series Split        14  12.625416  8.583012  11.250783   
3              Repeated KFold         1   2.447537  0.102006   1.929530   
4              Repeated KFold         7   2.543129  0.269894   1.978938   
5              Repeated KFold        14   2.705009  0.254875   2.102688   

            MAPE Test              R2 Test             
        std      mean       std       mean        std  
0  2.161510  0.217044  0.056261 -15.613525   8.335682  
1  3.255173  0.184884  0.087198 -22.762381  32.120369  
2  8.124490  0.268252  0.191581 -69.727838  74.197580  
3  0.085710  0.044662  0.001983   0.510017   0.063026  
4  0.161019  0.046151  0.003689   0.449090   0.145756  
5  0.155796  0.049098  0.003892   0.329255   0.150663

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [32]:
set_seeds()
start = time.time()
gap = 0
n_splits = 5
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]
REPEAT = 0

print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()
    scores_final = None

    for repeat in range(repeats):
        REPEAT = repeat
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1)
        y = df_copy["CS28"]

        cv = TimeSeriesSplit(
            gap=gap, max_train_size=None, n_splits=n_splits, test_size=None
        )
        scores = custom_cross_validate_parallelized(
            LSTM,
            SimpleImputer,
            StandardScaler,
            x,
            y,
            cv,
            timesteps,
            dates=dates,
            cement_types=df_copy[CEMENT_TYPES],
            estimator_params=None,
            imputer_params={"strategy": "median"},
        )
        scores = scores[0]
        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    scores = {key: np.array(val).flatten() for key, val in scores_final.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Time Series Split"
    results_dict_copy[
        "Cross Validation Params"
    ] = '{"N_Splits": 5, "Repeats": 3, "Gap": 0}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(results_dict_copy, scores)
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Blocking Time Series Split:
Repeats: 3
n_splits: 5



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.872 (0.976)
MAE: 2.223 (0.689)
MAPE: 0.051 (0.016)
R2: 0.215 (0.507)


******
[TEST]
******
RMSE: 6.040 (4.488)
MAE: 4.641 (3.571)
MAPE: 0.108 (0.082)
R2: -5.064 (8.988)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.953 (0.414)
MAE: 1.590 (0.393)
MAPE: 0.036 (0.009)
R2: 0.636 (0.163)


******
[TEST]
******
RMSE: 7.272 (5.859)
MAE: 5.657 (4.661)
MAPE: 0.132 (0.107)
R2: -9.081 (17.053)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.733 (0.184)
MAE: 1.369 (0.125)
MAPE: 0.031 (0.003)
R2: 0.718 (0.061)


******
[TEST]
******
RMSE: 8.258 (6.520)
MAE: 6.276 (5.052)
MAPE: 0.147 (0.117)
R2: -12.700 (20.553)


Minutes Elapsed:  13.647944084803264


<IPython.core.display.Javascript object>

In [33]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features Model  \
                                                          
0  Chemical + Mineralogical + Feature Engineering  LSTM   
1  Chemical + Mineralogical + Feature Engineering  LSTM   
2  Chemical + Mineralogical + Feature Engineering  LSTM   
3  Chemical + Mineralogical + Feature Engineering  LSTM   
4  Chemical + Mineralogical + Feature Engineering  LSTM   
5  Chemical + Mineralogical + Feature Engineering  LSTM   
6  Chemical + Mineralogical + Feature Engineering  LSTM   
7  Chemical + Mineralogical + Feature Engineering  LSTM   
8  Chemical + Mineralogical + Feature Engineering  LSTM   

             Cross Validation Timesteps  RMSE Test             MAE Test  \
                                              mean       std       mean   
0  Blocking Time Series Split         1  12.299179  2.931981   9.335137   
1  Blocking Time Series Split         7   9.779360  3.836300   7.821624   
2  Blocking Time Series Split        14  12.625416  8.583012  11.250783   
3              Repeated KFold         1   2.447537  0.102006   1.929530   
4              Repeated KFold         7   2.543129  0.269894   1.978938   
5              Repeated KFold        14   2.705009  0.254875   2.102688   
6           Time Series Split         1   6.039704  4.487765   4.641301   
7           Time Series Split         7   7.272149  5.859386   5.656769   
8           Time Series Split        14   8.258194  6.519557   6.276429   

            MAPE Test              R2 Test             
        std      mean       std       mean        std  
0  2.161510  0.217044  0.056261 -15.613525   8.335682  
1  3.255173  0.184884  0.087198 -22.762381  32.120369  
2  8.124490  0.268252  0.191581 -69.727838  74.197580  
3  0.085710  0.044662  0.001983   0.510017   0.063026  
4  0.161019  0.046151  0.003689   0.449090   0.145756  
5  0.155796  0.049098  0.003892   0.329255   0.150663  
6  3.571049  0.107959  0.081605  -5.063682   8.988194  
7  4.661008  0.131616  0.106945  -9.080575  17.053223  
8  5.051780  0.147149  0.116902 -12.699687  20.552884

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [34]:
start = time.time()
test_size = 0.2
repeats = 3
TIMESTEPS_LIST = [1, 7, 14]

print("Out of time Cross Val:")
print(f"Repeats: {repeats}")
print(f"Train: {80}%", f"Test: {20}%")
print()

for timesteps in TIMESTEPS_LIST:
    set_seeds()

    def parallel_repeats(repeat):
        set_seeds(SEED + repeat)

        scores_final = None

        # Data Splitting
        x = df_copy.drop(["Date", "CS28"] + CEMENT_TYPES, axis=1).copy()
        y = df_copy["CS28"].copy()

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=test_size, random_state=SEED, shuffle=False
        )

        # Preprocessing
        imputer = SimpleImputer(strategy="median")
        scaler = StandardScaler()

        x_train = imputer.fit_transform(x_train)
        x_train = scaler.fit_transform(x_train)
        dates_train = dates[: x_train.shape[0]].reset_index(drop=True)
        cement_types_train = df_copy[CEMENT_TYPES][: x_train.shape[0]].reset_index(
            drop=True
        )

        x_test = imputer.transform(x_test)
        x_test = scaler.transform(x_test)
        dates_test = dates[x_train.shape[0] :].reset_index(drop=True)
        cement_types_test = df_copy[CEMENT_TYPES][x_train.shape[0] :].reset_index(
            drop=True
        )

        # Sequence Splitting
        data_train = pd.concat(
            [
                dates_train,
                pd.DataFrame(x_train, columns=x.columns),
                cement_types_train,
                y_train.reset_index(drop=True),
            ],
            axis=1,
        )
        data_test = pd.concat(
            [
                dates_test,
                pd.DataFrame(x_test, columns=x.columns),
                cement_types_test,
                y_test.reset_index(drop=True),
            ],
            axis=1,
        )

        x_train, y_train = split_sequences_per_cement_type(data_train, timesteps)
        x_test, y_test = split_sequences_per_cement_type(data_test, timesteps)

        # Train model and test evalutation
        # Fit model
        pipeline = Pipeline([("estimator", LSTM())])
        pipeline.fit(x_train, y_train)

        # Make predictions
        y_train_pred = pipeline.predict(x_train)
        y_test_pred = pipeline.predict(x_test)

        # evaluate predictions
        scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)

        if scores_final is None:
            scores_final = {key: [] for key, _ in scores.items()}

        for key, value in scores.items():
            scores_final[key] += [value]

        return scores_final

    scores = Parallel(n_jobs=5)(
        delayed(parallel_repeats)(repeat) for repeat in range(repeats)
    )
    scores_final = {key: [] for key, _ in scores[0].items()}
    for scores_dict in scores:
        for key, value in scores_dict.items():
            scores_final[key] += value

    print("TIMESTEPS: %d " % timesteps)
    print_scores(scores_final, METRICS, METRICS_DICT)

    # Saving the results
    # scores = {key: val[0] for key, val in scores.items()}
    results_dict_copy = results_dict.copy()
    results_dict_copy["Timesteps"] = timesteps
    results_dict_copy["Cross Validation"] = "Out of time Split"
    results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
    results_dict_copy["Data Shape"] = x.shape
    df_results = fill_results_dict(
        results_dict_copy, {key: value for key, value in scores_final.items()}
    )
    results_to_save.append(df_results)

end = time.time()
print("Minutes Elapsed: ", (end - start) / 60)

Out of time Cross Val:
Repeats: 3
Train: 80% Test: 20%



TIMESTEPS: 1 
******
[TRAIN]
******
RMSE: 2.155 (0.023)
MAE: 1.703 (0.029)
MAPE: 0.039 (0.001)
R2: 0.563 (0.009)


******
[TEST]
******
RMSE: 3.016 (0.129)
MAE: 2.406 (0.101)
MAPE: 0.062 (0.003)
R2: -0.176 (0.100)




TIMESTEPS: 7 
******
[TRAIN]
******
RMSE: 1.771 (0.102)
MAE: 1.424 (0.085)
MAPE: 0.032 (0.002)
R2: 0.698 (0.035)


******
[TEST]
******
RMSE: 3.557 (0.228)
MAE: 2.869 (0.208)
MAPE: 0.073 (0.005)
R2: -0.708 (0.219)




TIMESTEPS: 14 
******
[TRAIN]
******
RMSE: 1.738 (0.047)
MAE: 1.385 (0.035)
MAPE: 0.032 (0.001)
R2: 0.708 (0.016)


******
[TEST]
******
RMSE: 3.650 (0.471)
MAE: 3.073 (0.487)
MAPE: 0.079 (0.013)
R2: -0.918 (0.475)


Minutes Elapsed:  4.958346962928772


<IPython.core.display.Javascript object>

In [35]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation", "Timesteps"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)


Features Model  \
                                                           
0   Chemical + Mineralogical + Feature Engineering  LSTM   
1   Chemical + Mineralogical + Feature Engineering  LSTM   
2   Chemical + Mineralogical + Feature Engineering  LSTM   
3   Chemical + Mineralogical + Feature Engineering  LSTM   
4   Chemical + Mineralogical + Feature Engineering  LSTM   
5   Chemical + Mineralogical + Feature Engineering  LSTM   
6   Chemical + Mineralogical + Feature Engineering  LSTM   
7   Chemical + Mineralogical + Feature Engineering  LSTM   
8   Chemical + Mineralogical + Feature Engineering  LSTM   
9   Chemical + Mineralogical + Feature Engineering  LSTM   
10  Chemical + Mineralogical + Feature Engineering  LSTM   
11  Chemical + Mineralogical + Feature Engineering  LSTM   

              Cross Validation Timesteps  RMSE Test             MAE Test  \
                                               mean       std       mean   
0   Blocking Time Series Split         1  12.299179  2.931981   9.335137   
1   Blocking Time Series Split         7   9.779360  3.836300   7.821624   
2   Blocking Time Series Split        14  12.625416  8.583012  11.250783   
3            Out of time Split         1   3.016232  0.128554   2.406490   
4            Out of time Split         7   3.556740  0.228020   2.869203   
5            Out of time Split        14   3.650371  0.471322   3.073400   
6               Repeated KFold         1   2.447537  0.102006   1.929530   
7               Repeated KFold         7   2.543129  0.269894   1.978938   
8               Repeated KFold        14   2.705009  0.254875   2.102688   
9            Time Series Split         1   6.039704  4.487765   4.641301   
10           Time Series Split         7   7.272149  5.859386   5.656769   
11           Time Series Split        14   8.258194  6.519557   6.276429   

             MAPE Test              R2 Test             
         std      mean       std       mean        std  
0   2.161510  0.217044  0.056261 -15.613525   8.335682  
1   3.255173  0.184884  0.087198 -22.762381  32.120369  
2   8.124490  0.268252  0.191581 -69.727838  74.197580  
3   0.101310  0.061687  0.002615  -0.176465   0.099862  
4   0.208479  0.073138  0.005181  -0.707910   0.219131  
5   0.487316  0.079400  0.012606  -0.918145   0.474539  
6   0.085710  0.044662  0.001983   0.510017   0.063026  
7   0.161019  0.046151  0.003689   0.449090   0.145756  
8   0.155796  0.049098  0.003892   0.329255   0.150663  
9   3.571049  0.107959  0.081605  -5.063682   8.988194  
10  4.661008  0.131616  0.106945  -9.080575  17.053223  
11  5.051780  0.147149  0.116902 -12.699687  20.552884

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [36]:
path = "../../../../../../reports/results/local_models/partner_ii/all_cements/full/"
filename = f"lstm_results_full_{index_to_save}.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [37]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../../reports/results/local_models/partner_ii/all_cements/grouped/"
filename = f"lstm_results_grouped_{index_to_save}.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>